# LTX Video and OpenVINO™

[LTX-Video](https://github.com/Lightricks/LTX-Video) is a transformer-based latent diffusion model that adopts a holistic approach to video generation by seamlessly integrating the responsibilities of the Video-VAE and the denoising transformer. Unlike existing methods, which treat these components as independent, LTX-Video aims to optimize their interaction for improved efficiency and quality. At its core is a carefully designed Video-VAE that achieves a high compression ratio of 1:192, with spatiotemporal downscaling of 32×32×8 pixels per token, enabled by relocating the patchifying operation from the transformer’s input to the VAE's input. Operating in this highly compressed latent space enables the transformer to efficiently perform full spatiotemporal selfattention, which is essential for generating high-resolution videos with temporal consistency. However, the high compression inherently limits the representation of fine details. To address this, this VAE decoder is tasked with both latent-to-pixel conversion and the final denoising step, producing the clean result directly in pixel space. This approach preserves the ability to generate fine details without incurring the runtime cost of a separate upsampling module. The model supports diverse use cases, including text-to-video and image-to-video generation, with both capabilities
trained simultaneously.

In this example we show how to convert text-to-video pipeline in OpenVINO format and run inference. An additional part demonstrates how to run optimization with [NNCF](https://github.com/openvinotoolkit/nncf/) to speed up pipeline.

| | | | |
|:---:|:---:|:---:|:---:|
| ![example1](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main/docs/_static/ltx-video_example_00001.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A woman with long brown hair and light skin smiles at another woman...</summary>A woman with long brown hair and light skin smiles at another woman with long blonde hair. The woman with brown hair wears a black jacket and has a small, barely noticeable mole on her right cheek. The camera angle is a close-up, focused on the woman with brown hair's face. The lighting is warm and natural, likely from the setting sun, casting a soft glow on the scene. The scene appears to be real-life footage.</details> | ![example2](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00002.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A woman walks away from a white Jeep parked on a city street at night...</summary>A woman walks away from a white Jeep parked on a city street at night, then ascends a staircase and knocks on a door. The woman, wearing a dark jacket and jeans, walks away from the Jeep parked on the left side of the street, her back to the camera; she walks at a steady pace, her arms swinging slightly by her sides; the street is dimly lit, with streetlights casting pools of light on the wet pavement; a man in a dark jacket and jeans walks past the Jeep in the opposite direction; the camera follows the woman from behind as she walks up a set of stairs towards a building with a green door; she reaches the top of the stairs and turns left, continuing to walk towards the building; she reaches the door and knocks on it with her right hand; the camera remains stationary, focused on the doorway; the scene is captured in real-life footage.</details> | ![example3](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00003.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A woman with blonde hair styled up, wearing a black dress...</summary>A woman with blonde hair styled up, wearing a black dress with sequins and pearl earrings, looks down with a sad expression on her face. The camera remains stationary, focused on the woman's face. The lighting is dim, casting soft shadows on her face. The scene appears to be from a movie or TV show.</details> | ![example4](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00004.gif)<br><details style="max-width: 300px; margin: auto;"><summary>The camera pans over a snow-covered mountain range...</summary>The camera pans over a snow-covered mountain range, revealing a vast expanse of snow-capped peaks and valleys.The mountains are covered in a thick layer of snow, with some areas appearing almost white while others have a slightly darker, almost grayish hue. The peaks are jagged and irregular, with some rising sharply into the sky while others are more rounded. The valleys are deep and narrow, with steep slopes that are also covered in snow. The trees in the foreground are mostly bare, with only a few leaves remaining on their branches. The sky is overcast, with thick clouds obscuring the sun. The overall impression is one of peace and tranquility, with the snow-covered mountains standing as a testament to the power and beauty of nature.</details> |
| ![example5](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00005.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A woman with light skin, wearing a blue jacket and a black hat...</summary>A woman with light skin, wearing a blue jacket and a black hat with a veil, looks down and to her right, then back up as she speaks; she has brown hair styled in an updo, light brown eyebrows, and is wearing a white collared shirt under her jacket; the camera remains stationary on her face as she speaks; the background is out of focus, but shows trees and people in period clothing; the scene is captured in real-life footage.</details> | ![example6](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00006.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A man in a dimly lit room talks on a vintage telephone...</summary>A man in a dimly lit room talks on a vintage telephone, hangs up, and looks down with a sad expression. He holds the black rotary phone to his right ear with his right hand, his left hand holding a rocks glass with amber liquid. He wears a brown suit jacket over a white shirt, and a gold ring on his left ring finger. His short hair is neatly combed, and he has light skin with visible wrinkles around his eyes. The camera remains stationary, focused on his face and upper body. The room is dark, lit only by a warm light source off-screen to the left, casting shadows on the wall behind him. The scene appears to be from a movie.</details> | ![example7](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00007.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A prison guard unlocks and opens a cell door...</summary>A prison guard unlocks and opens a cell door to reveal a young man sitting at a table with a woman. The guard, wearing a dark blue uniform with a badge on his left chest, unlocks the cell door with a key held in his right hand and pulls it open; he has short brown hair, light skin, and a neutral expression. The young man, wearing a black and white striped shirt, sits at a table covered with a white tablecloth, facing the woman; he has short brown hair, light skin, and a neutral expression. The woman, wearing a dark blue shirt, sits opposite the young man, her face turned towards him; she has short blonde hair and light skin. The camera remains stationary, capturing the scene from a medium distance, positioned slightly to the right of the guard. The room is dimly lit, with a single light fixture illuminating the table and the two figures. The walls are made of large, grey concrete blocks, and a metal door is visible in the background. The scene is captured in real-life footage.</details> | ![example8](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00008.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A woman with blood on her face and a white tank top...</summary>A woman with blood on her face and a white tank top looks down and to her right, then back up as she speaks. She has dark hair pulled back, light skin, and her face and chest are covered in blood. The camera angle is a close-up, focused on the woman's face and upper torso. The lighting is dim and blue-toned, creating a somber and intense atmosphere. The scene appears to be from a movie or TV show.</details> |
| ![example9](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00009.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A man with graying hair, a beard, and a gray shirt...</summary>A man with graying hair, a beard, and a gray shirt looks down and to his right, then turns his head to the left. The camera angle is a close-up, focused on the man's face. The lighting is dim, with a greenish tint. The scene appears to be real-life footage. Step</details> | ![example10](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00010.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A clear, turquoise river flows through a rocky canyon...</summary>A clear, turquoise river flows through a rocky canyon, cascading over a small waterfall and forming a pool of water at the bottom.The river is the main focus of the scene, with its clear water reflecting the surrounding trees and rocks. The canyon walls are steep and rocky, with some vegetation growing on them. The trees are mostly pine trees, with their green needles contrasting with the brown and gray rocks. The overall tone of the scene is one of peace and tranquility.</details> | ![example11](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00011.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A man in a suit enters a room and speaks to two women...</summary>A man in a suit enters a room and speaks to two women sitting on a couch. The man, wearing a dark suit with a gold tie, enters the room from the left and walks towards the center of the frame. He has short gray hair, light skin, and a serious expression. He places his right hand on the back of a chair as he approaches the couch. Two women are seated on a light-colored couch in the background. The woman on the left wears a light blue sweater and has short blonde hair. The woman on the right wears a white sweater and has short blonde hair. The camera remains stationary, focusing on the man as he enters the room. The room is brightly lit, with warm tones reflecting off the walls and furniture. The scene appears to be from a film or television show.</details> | ![example12](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00012.gif)<br><details style="max-width: 300px; margin: auto;"><summary>The waves crash against the jagged rocks of the shoreline...</summary>The waves crash against the jagged rocks of the shoreline, sending spray high into the air.The rocks are a dark gray color, with sharp edges and deep crevices. The water is a clear blue-green, with white foam where the waves break against the rocks. The sky is a light gray, with a few white clouds dotting the horizon.</details> |
| ![example13](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00013.gif)<br><details style="max-width: 300px; margin: auto;"><summary>The camera pans across a cityscape of tall buildings...</summary>The camera pans across a cityscape of tall buildings with a circular building in the center. The camera moves from left to right, showing the tops of the buildings and the circular building in the center. The buildings are various shades of gray and white, and the circular building has a green roof. The camera angle is high, looking down at the city. The lighting is bright, with the sun shining from the upper left, casting shadows from the buildings. The scene is computer-generated imagery.</details> | ![example14](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00014.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A man walks towards a window, looks out, and then turns around...</summary>A man walks towards a window, looks out, and then turns around. He has short, dark hair, dark skin, and is wearing a brown coat over a red and gray scarf. He walks from left to right towards a window, his gaze fixed on something outside. The camera follows him from behind at a medium distance. The room is brightly lit, with white walls and a large window covered by a white curtain. As he approaches the window, he turns his head slightly to the left, then back to the right. He then turns his entire body to the right, facing the window. The camera remains stationary as he stands in front of the window. The scene is captured in real-life footage.</details> | ![example15](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00015.gif)<br><details style="max-width: 300px; margin: auto;"><summary>Two police officers in dark blue uniforms and matching hats...</summary>Two police officers in dark blue uniforms and matching hats enter a dimly lit room through a doorway on the left side of the frame. The first officer, with short brown hair and a mustache, steps inside first, followed by his partner, who has a shaved head and a goatee. Both officers have serious expressions and maintain a steady pace as they move deeper into the room. The camera remains stationary, capturing them from a slightly low angle as they enter. The room has exposed brick walls and a corrugated metal ceiling, with a barred window visible in the background. The lighting is low-key, casting shadows on the officers' faces and emphasizing the grim atmosphere. The scene appears to be from a film or television show.</details> | ![example16](https://media.githubusercontent.com/media/Lightricks/LTX-Video/refs/heads/main//docs/_static/ltx-video_example_00016.gif)<br><details style="max-width: 300px; margin: auto;"><summary>A woman with short brown hair, wearing a maroon sleeveless top...</summary>A woman with short brown hair, wearing a maroon sleeveless top and a silver necklace, walks through a room while talking, then a woman with pink hair and a white shirt appears in the doorway and yells. The first woman walks from left to right, her expression serious; she has light skin and her eyebrows are slightly furrowed. The second woman stands in the doorway, her mouth open in a yell; she has light skin and her eyes are wide. The room is dimly lit, with a bookshelf visible in the background. The camera follows the first woman as she walks, then cuts to a close-up of the second woman's face. The scene is captured in real-life footage.</details> |

#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Load and run the original model](#Load-the-original-model)
- [Convert the model to OpenVINO IR](#Convert-the-model-to-OpenVINO-IR)
- [Compiling the model](#Compiling-the-model)
- [Run OpenVINO model inference](#Run-OpenVINO-model-inference)
- [Quantization](#Quantization)
    - [Prepare calibration dataset](#Prepare-calibration-dataset)
    - [Run Quantization](#Run-Quantization)
    - [Run Quantized OpenVINO model inference](#Run-Quantized-OpenVINO-model-inference)
    - [Compare Transformer File Size](#Compare-Transformer-File-Size)
    - [Compare inference time of the FP16 IR and quantized models](#Compare-inference-time-of-the-FP16-IR-and-quantized-models)
- [Interactive inference](#Interactive-inference)

### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/ltx-video/ltx-video.ipynb" />

## Prerequisites
[back to top ⬆️](#Table-of-contents:)

In [ ]:
%pip install -qU "torch>=2.1.0" "torchvision>=0.16" "diffusers>=0.28.2" "transformers>=4.44.2" "sentencepiece>=0.1.96" "huggingface-hub~=0.25.2" "einops" "accelerate" "matplotlib" "imageio[ffmpeg]" "nncf>=2.15" "gradio>=4.26" --extra-index-url https://download.pytorch.org/whl/cpu
%pip install --pre -qU "openvino>=2025.0.0" --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly

In [ ]:
from pathlib import Path
import requests


if not Path("notebook_utils.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
    )
    open("notebook_utils.py", "w").write(r.text)


if not Path("ov_ltx_video_helper.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/ltx-video/ov_ltx_video_helper.py",
    )
    open("ov_ltx_video_helper.py", "w").write(r.text)


# Read more about telemetry collection at https://github.com/openvinotoolkit/openvino_notebooks?tab=readme-ov-file#-telemetry
from notebook_utils import collect_telemetry

collect_telemetry("ltx-video.ipynb")

### Load the original model
[back to top ⬆️](#Table-of-contents:)

In [ ]:
import torch
from diffusers import LTXPipeline
from diffusers.utils import export_to_video

pipe = LTXPipeline.from_pretrained("Lightricks/LTX-Video")

### Convert the model to OpenVINO IR
[back to top ⬆️](#Table-of-contents:)

 
OpenVINO supports PyTorch models via conversion to OpenVINO Intermediate Representation (IR). [OpenVINO model conversion API](https://docs.openvino.ai/2024/openvino-workflow/model-preparation.html#convert-a-model-with-python-convert-model) should be used for these purposes. `ov.convert_model` function accepts original PyTorch model instance and example input for tracing and returns `ov.Model` representing this model in OpenVINO framework. Converted model can be used for saving on disk using `ov.save_model` function or directly loading on device using `core.complie_model`.

`ov_ltx_video_helper.py` script contains helper function for models downloading and models conversion, please check its content if you interested in conversion details. Note that we delete the original models after conversion from pipeline to free memory.

LTX Video text-to-video pipeline consists of 3 models: `Text Encoder` converts input text into embeddings, `Transformer` processes these embeddings to generate latents from noise step by step, `VAEDecoder` performs the last denoising step in conjunction with converting latents to pixels.

In [4]:
from ov_ltx_video_helper import convert_text_encoder, convert_transformer, convert_vae_decoder


# Uncomment line below to see model conversion code (replace to convert_transformer and convert_vae_decoder to see them too)
# convert_text_encoder??

In [ ]:
import gc


MODEL_DIR = Path("models")


TEXT_ENCODER_PATH = MODEL_DIR / "text_encoder_ir.xml"
TRANSFORMER_OV_PATH = MODEL_DIR / "transformer_ir.xml"
VAE_DECODER_PATH = MODEL_DIR / "vae_ir.xml"

text_encoder_dtype = pipe.text_encoder.dtype
transformer_config = pipe.transformer.config
vae_config = pipe.vae.config
vae_latents_mean = pipe.vae.latents_mean
vae_latents_std = pipe.vae.latents_std

convert_text_encoder(pipe.text_encoder, TEXT_ENCODER_PATH)
del pipe.text_encoder
convert_transformer(pipe.transformer, TRANSFORMER_OV_PATH)
del pipe.transformer
convert_vae_decoder(pipe.vae, VAE_DECODER_PATH)
del pipe.vae
gc.collect()

### Compiling the model
[back to top ⬆️](#Table-of-contents:)

Select device from dropdown list for running inference using OpenVINO.

In [6]:
import openvino as ov

from notebook_utils import device_widget


core = ov.Core()

device = device_widget()

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

`ov_catvton_helper.py` provides wrapper classes that wrap the compiled models to keep the original interface. Note that all of wrapper classes return `torch.Tensor`s instead of `np.array`s. Then we insert wrappers instances in the pipeline. 

In [7]:
from ov_ltx_video_helper import TextEncoderWrapper, ConvTransformerWrapper, VAEWrapper


# Uncomment the line below to see the wrapper class code (replace to ConvTransformerWrapper and VAEWrapper to see them too)
# TextEncoderWrapper??

In [ ]:
compiled_transformer = core.compile_model(TRANSFORMER_OV_PATH, device.value)
compiled_vae = core.compile_model(VAE_DECODER_PATH, device.value)
compiled_text_encoder = core.compile_model(TEXT_ENCODER_PATH, device.value)

pipe.__dict__["_internal_dict"]["_execution_device"] = pipe._execution_device  # this is to avoid some problem that can occur in the pipeline

pipe.register_modules(
    text_encoder=TextEncoderWrapper(compiled_text_encoder, text_encoder_dtype),
    transformer=ConvTransformerWrapper(compiled_transformer, transformer_config),
    vae=VAEWrapper(compiled_vae, vae_config, vae_latents_mean, vae_latents_std),
)

### Run OpenVINO model inference
[back to top ⬆️](#Table-of-contents:)

[General tips](https://huggingface.co/Lightricks/LTX-Video#general-tips):
- The model works on resolutions that are divisible by 32 and number of frames that are divisible by 8 + 1 (e.g. 257). In case the input is not satisfied to the described conditions, the input will be padded with -1 and then cropped to the desired resolution and number of frames.
- The model works best on resolutions under 720 x 1280 and number of frames below 257.
- Prompts should be in English. The more elaborate the better. Good prompt looks like The turquoise waves crash against the dark, jagged rocks of the shore, sending white foam spraying into the air. The scene is dominated by the stark contrast between the bright blue water and the dark, almost black rocks. The water is a clear, turquoise color, and the waves are capped with white foam. The rocks are dark and jagged, and they are covered in patches of green moss. The shore is lined with lush green vegetation, including trees and bushes. In the background, there are rolling hills covered in dense forest. The sky is cloudy, and the light is dim.

In [9]:
prompt = "A clear, turquoise river flows through a rocky canyon, cascading over a small waterfall and forming a pool of water at the bottom.The river is the main focus of the scene, with its clear water reflecting the surrounding trees and rocks. The canyon walls are steep and rocky, with some vegetation growing on them. The trees are mostly pine trees, with their green needles contrasting with the brown and gray rocks. The overall tone of the scene is one of peace and tranquility."
negative_prompt = "worst quality, inconsistent motion, blurry, jittery, distorted"
generator = torch.Generator(device="cpu").manual_seed(42)


video = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=704,
    height=480,
    num_frames=24,
    num_inference_steps=40,
    generator=generator,
    guidance_scale=3,
).frames[0]
export_to_video(video, "output_ov.mp4", fps=24)

/home/maleksandr/test_notebooks/sd-xl/openvino_notebooks/venv/lib/python3.10/site-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `_execution_device` directly via 'LTXPipeline' object attribute is deprecated. Please access '_execution_device' over 'LTXPipeline's config object instead, e.g. 'scheduler.config._execution_device'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/40 [00:00<?, ?it/s]

'output_ov.mp4'

In [10]:
from IPython.display import Video

Video("output_ov.mp4")

## Quantization
[back to top ⬆️](#Table-of-contents:)

[NNCF](https://github.com/openvinotoolkit/nncf/) enables post-training quantization by adding quantization layers into model graph and then using a subset of the training dataset to initialize the parameters of these additional quantization layers. Quantized operations are executed in `INT8` instead of `FP32`/`FP16` making model inference faster.

According to `LTX Video Model` structure, the transformer model takes up significant portion of the overall pipeline execution time. Now we will show you how to optimize the transformer part using [NNCF](https://github.com/openvinotoolkit/nncf/) to reduce computation cost and speed up the pipeline. Quantizing the rest of the LTX Video pipeline does not significantly improve inference performance but can lead to a substantial degradation of accuracy.

For the transformer model we apply quantization in hybrid mode which means that we quantize: (1) weights of MatMul layers and (2) activations of other layers. The steps are the following:

1. Create a calibration dataset for quantization.
2. Collect operations with weights.
3. Run `nncf.compress_model()` to compress only the model weights.
4. Run `nncf.quantize()` on the compressed model with weighted operations ignored by providing `ignored_scope` parameter.
5. Save the `INT8` model using `openvino.save_model()` function.

Please select below whether you would like to run quantization to improve model inference speed.

> **NOTE**: Quantization is time and memory consuming operation. Running quantization code below may take some time.

In [11]:
from notebook_utils import quantization_widget

to_quantize = quantization_widget()

to_quantize

Checkbox(value=True, description='Quantization')

In [12]:
# Fetch `skip_kernel_extension` module
import requests

if not Path("skip_kernel_extension.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/skip_kernel_extension.py",
    )
    open("skip_kernel_extension.py", "w").write(r.text)

int8_pipe = None

core = ov.Core()

%load_ext skip_kernel_extension

### Prepare calibration dataset
[back to top ⬆️](#Table-of-contents:)

In [13]:
%%skip not $to_quantize.value

from ltx_video_quantization_helper import TRANSFORMER_INT8_PATH, collect_calibration_data


if not TRANSFORMER_INT8_PATH.exists():
    calibration_dataset_size = 200
    print("Calibration data collection started")
    transformer_calibration_data = collect_calibration_data(
        pipe,
        calibration_dataset_size=calibration_dataset_size,
        num_inference_steps=30,
        guidance_scale=5
    )
    print("Calibration data collection finished")

Calibration data collection started
Calibration data collection finished


### Run Quantization
[back to top ⬆️](#Table-of-contents:)

In [14]:
%%skip not $to_quantize.value


import nncf
import logging
from nncf.quantization.advanced_parameters import AdvancedSmoothQuantParameters

from ltx_video_quantization_helper import get_operation_const_op, collect_ops_with_weights


if not TRANSFORMER_INT8_PATH.exists():
    del pipe
    gc.collect()

    transformer_model = core.read_model(TRANSFORMER_OV_PATH)
    transformer_ignored_scope = collect_ops_with_weights(transformer_model)
    compressed_transformer_model = nncf.compress_weights(transformer_model)
    quantized_transformer_model = nncf.quantize(
        model=compressed_transformer_model,
        calibration_dataset=nncf.Dataset(transformer_calibration_data),
        subset_size=calibration_dataset_size,
        model_type=nncf.ModelType.TRANSFORMER,
        ignored_scope=nncf.IgnoredScope(names=transformer_ignored_scope),
        advanced_parameters=nncf.AdvancedQuantizationParameters(smooth_quant_alphas=AdvancedSmoothQuantParameters(matmul=-1))
    )
    ov.save_model(quantized_transformer_model, TRANSFORMER_INT8_PATH)

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_asym                 │ 100% (287 / 287)            │ 100% (287 / 287)                       │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

INFO:nncf:287 ignored nodes were found by names in the NNCFGraph
INFO:nncf:Not adding activation input quantizer for operation: 10 __module.caption_projection.linear_1/aten::linear/MatMul
21 __module.caption_projection.linear_1/aten::linear/Add

INFO:nncf:Not adding activation input quantizer for operation: 61 __module.caption_projection.linear_2/aten::linear/MatMul
167 __module.caption_projection.linear_2/aten::linear/Add

INFO:nncf:Not adding activation input quantizer for operation: 104 __module.transformer_blocks.0.attn2.to_k/aten::linear/MatMul
239 __module.transformer_blocks.0.attn2.to_k/aten::linear/Add

INFO:nncf:Not adding activation input quantizer for operation: 105 __module.transformer_blocks.0.attn2.to_v/aten::linear/MatMul
240 __module.transformer_blocks.0.attn2.to_v/aten::linear/Add

INFO:nncf:Not adding activation input quantizer for operation: 106 __module.transformer_blocks.1.attn2.to_k/aten::linear/MatMul
241 __module.transformer_blocks.1.attn2.to_k/aten::linear/Add


Output()

In [15]:
%%skip not $to_quantize.value

pipe = LTXPipeline.from_pretrained("Lightricks/LTX-Video")

compiled_transformer = core.compile_model(TRANSFORMER_INT8_PATH, device.value)
compiled_vae = core.compile_model(VAE_DECODER_PATH, device.value)
compiled_text_encoder = core.compile_model(TEXT_ENCODER_PATH, device.value)

pipe.__dict__["_internal_dict"]["_execution_device"] = pipe._execution_device  # this is to avoid some problem that can occur in the pipeline

pipe.register_modules(
    text_encoder=TextEncoderWrapper(compiled_text_encoder, text_encoder_dtype),
    transformer=ConvTransformerWrapper(compiled_transformer, transformer_config),
    vae=VAEWrapper(compiled_vae, vae_config, vae_latents_mean, vae_latents_std),
)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Run Quantized OpenVINO model inference
[back to top ⬆️](#Table-of-contents:)

In [16]:
%%skip not $to_quantize.value

prompt = "A clear, turquoise river flows through a rocky canyon, cascading over a small waterfall and forming a pool of water at the bottom.The river is the main focus of the scene, with its clear water reflecting the surrounding trees and rocks. The canyon walls are steep and rocky, with some vegetation growing on them. The trees are mostly pine trees, with their green needles contrasting with the brown and gray rocks. The overall tone of the scene is one of peace and tranquility."
negative_prompt = "worst quality, inconsistent motion, blurry, jittery, distorted"
generator = torch.Generator(device="cpu").manual_seed(42)


video = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=704,
    height=480,
    num_frames=24,
    num_inference_steps=40,
    generator=generator,
    guidance_scale=3,
).frames[0]
export_to_video(video, "output_int8_ov.mp4", fps=24)

/home/maleksandr/test_notebooks/sd-xl/openvino_notebooks/venv/lib/python3.10/site-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `_execution_device` directly via 'LTXPipeline' object attribute is deprecated. Please access '_execution_device' over 'LTXPipeline's config object instead, e.g. 'scheduler.config._execution_device'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/40 [00:00<?, ?it/s]

In [17]:
%%skip not $to_quantize.value

from IPython.display import Video

display(Video("output_int8_ov.mp4"))

### Compare Transformer File Size
[back to top ⬆️](#Table-of-contents:)

In [ ]:
%%skip not $to_quantize.value


from pathlib import Path

fp16_ir_model_size = Path(TRANSFORMER_OV_PATH).with_suffix(".bin").stat().st_size / 1024 / 1024
quantized_model_size = Path(TRANSFORMER_INT8_PATH).with_suffix(".bin").stat().st_size / 1024 / 1024
print(f"FP16 IR model size: {fp16_ir_model_size:.2f} MB")
print(f"INT8 model size: {quantized_model_size:.2f} MB")
print(f"Model compression rate: {fp16_ir_model_size / quantized_model_size:.3f}")

FP16 IR model size: 3668.57 MB
INT8 model size: 1837.77 MB
Model compression rate: 1.996


### Compare inference time of the FP16 IR and quantized models
[back to top ⬆️](#Table-of-contents:)
To measure the inference performance of the `FP16` and `INT8` models, we use median inference time on calibration dataset.
So we can approximately estimate the speed up of the dynamic quantized models.


> **NOTE**: For the most accurate performance estimation, it is recommended to run `benchmark_app` in a terminal/command prompt after closing other applications with static shapes.

In [19]:
%%skip not $to_quantize.value

import time

import numpy as np


def calculate_inference_time(model_path, calibration_data):
    model = core.compile_model(model_path, device.value)
    inference_time = []
    for batch in calibration_data:
        start = time.perf_counter()
        _ = model(batch)[0]
        end = time.perf_counter()
        delta = end - start
        inference_time.append(delta)
    return np.median(inference_time)

In [20]:
%%skip not $to_quantize.value

fp16_latency = calculate_inference_time(TRANSFORMER_OV_PATH, transformer_calibration_data)
int8_latency = calculate_inference_time(TRANSFORMER_INT8_PATH, transformer_calibration_data)
print(f"Performance speed up: {fp16_latency / int8_latency:.3f}")

Performance speed up: 1.740


## Interactive inference
[back to top ⬆️](#Table-of-contents:)

Please select below whether you would like to use the quantized models to launch the interactive demo.

In [21]:
import ipywidgets as widgets

use_quantized_model = widgets.Checkbox(
    description="Use quantized model",
    value=TRANSFORMER_INT8_PATH.exists(),
    disabled=not TRANSFORMER_INT8_PATH.exists(),
)

use_quantized_model

Checkbox(value=True, description='Use quantized model')

In [ ]:
pipe = LTXPipeline.from_pretrained("Lightricks/LTX-Video")

compiled_transformer = (
    core.compile_model(TRANSFORMER_INT8_PATH, device.value) if use_quantized_model.value else core.compile_model(TRANSFORMER_OV_PATH, device.value)
)
compiled_vae = core.compile_model(VAE_DECODER_PATH, device.value)
compiled_text_encoder = core.compile_model(TEXT_ENCODER_PATH, device.value)

pipe.__dict__["_internal_dict"]["_execution_device"] = pipe._execution_device  # this is to avoid some problem that can occur in the pipeline

pipe.register_modules(
    text_encoder=TextEncoderWrapper(compiled_text_encoder, text_encoder_dtype),
    transformer=ConvTransformerWrapper(compiled_transformer, transformer_config),
    vae=VAEWrapper(compiled_vae, vae_config, vae_latents_mean, vae_latents_std),
)

In [23]:
import gradio as gr


def generate(prompt, negative_prompt, width, height, num_frames, num_inference_steps, seed, _=gr.Progress(track_tqdm=True)):
    generator = torch.Generator().manual_seed(seed)
    video = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_frames=num_frames,
        num_inference_steps=num_inference_steps,
        generator=generator,
    ).frames[0]
    file_name = "output.mp4"
    export_to_video(video, file_name, fps=24)
    return file_name

In [ ]:
if not Path("gradio_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/ltx-video/gradio_helper.py")
    open("gradio_helper.py", "w").write(r.text)

from gradio_helper import make_demo

demo = make_demo(fn=generate)

try:
    demo.queue().launch(debug=True)
except Exception:
    demo.queue().launch(share=True, debug=True)
# If you are launching remotely, specify server_name and server_port
# EXAMPLE: `demo.launch(server_name='your server name', server_port='server port in int')`
# To learn more please refer to the Gradio docs: https://gradio.app/docs/